## 二维卷积代码演示

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from math import floor

定义常量

In [2]:
in_channels = 1
out_channels = 1
kernel_size = 3
batch_size = 1
bias = False
input_size = [batch_size, in_channels, 4, 4]

In [3]:
conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, bias=bias)
input_feature_map = torch.randn(input_size)
output_feature_map = conv_layer(input_feature_map)

print(input_feature_map)
print(conv_layer.weight)
print('conv shape: ', conv_layer.weight.shape)  # out_channels * in_channels * height * width
print(output_feature_map)

tensor([[[[-0.6196, -2.0041,  1.1195,  0.7799],
          [ 0.3707,  0.4966, -0.3549,  0.5802],
          [-1.4829,  0.1418, -0.6850,  1.8667],
          [ 1.1718,  1.0255, -1.6357,  1.3332]]]])
Parameter containing:
tensor([[[[ 0.2092,  0.2278, -0.0349],
          [-0.1058, -0.3263,  0.2733],
          [ 0.1064,  0.0666, -0.1739]]]], requires_grad=True)
conv shape:  torch.Size([1, 1, 3, 3])
tensor([[[[-0.9527, -0.3249],
          [ 0.6040,  0.4898]]]], grad_fn=<ConvolutionBackward0>)


In [4]:
output_feature_map1  = F.conv2d(input_feature_map, conv_layer.weight)
print(output_feature_map1)

tensor([[[[-0.9527, -0.3249],
          [ 0.6040,  0.4898]]]], grad_fn=<ConvolutionBackward0>)


### 实现卷积

In [15]:
input = torch.randn(5, 5)  # 卷积输入特征图
kernel = torch.randn(3, 3)  # 卷积核
bias = torch.randn(1)  # 卷积偏置 默认输出通道数目为1

print(input)
print(kernel)
print(bias)

tensor([[ 0.0986, -1.2195, -1.5816,  0.0463, -1.2453],
        [ 0.0985, -0.5343,  0.8175, -0.7282, -0.3857],
        [-0.5309, -0.0902, -0.1461, -0.7005,  3.0180],
        [-0.4246,  0.8115, -0.3002,  0.1562,  0.8089],
        [ 1.2668,  0.5935, -2.5263, -0.0135, -0.0775]])
tensor([[ 0.6461, -1.2289, -0.1575],
        [ 0.8391, -1.7176, -0.4441],
        [-0.7126, -1.7188,  0.0099]])
tensor([-1.2915])


用原始的矩阵运算实现二维卷积

先不考虑batch_size维度和channel维度

In [16]:
def matrix_multiplication_for_conv2d(input, kernel, bias=0, stride=1, padding=0):
    if padding > 0:
        input = F.pad(input, (padding, padding, padding, padding))

    input_h, input_w = input.shape
    kernel_h, kernel_w = kernel.shape

    output_h = floor((input_h - kernel_h) / stride + 1)  # 卷积输入的高度
    output_w = floor((input_w - kernel_w) / stride + 1)  # 卷积输入的宽度
    output = torch.zeros(output_h, output_w)  # 初始化输出矩阵
    for i in range(0, input_h - kernel_h + 1, stride):  # 对高度进行遍历
        for j in range(0, input_w - kernel_w + 1, stride):  # 对宽度进行遍历
            region = input[i:i + kernel_h, j:j + kernel_h]  # 取出被核滑动到的区域
            output[int(i / stride), int(j / stride)] = torch.sum(region * kernel) + bias  # 点乘 并赋值给输出位置的元素

    return output

# 矩阵运算实现卷积的结果
mat_mul_conv_output = matrix_multiplication_for_conv2d(input, kernel, bias=bias, padding=1)
print(mat_mul_conv_output)

# 调用pytorch api卷积的结果
pytorch_api_conv_output = F.conv2d(input.reshape(1, 1, input.shape[0], input.shape[1]),
                                   kernel.reshape(1, 1, kernel.shape[0], kernel.shape[1]),
                                   padding=1,
                                   bias=bias)
print(pytorch_api_conv_output.squeeze())

torch.isclose(mat_mul_conv_output, pytorch_api_conv_output)  # 验证成功

tensor([[-1.0939,  2.4447, -0.6503, -1.4798,  2.0682],
        [-0.2408,  1.6893, -1.3638,  1.2721, -4.3678],
        [ 0.3614, -2.0208, -2.1008, -0.1139, -8.5612],
        [-2.4275, -5.0651,  3.9865, -0.0571, -6.5682],
        [-3.3370, -1.3500,  4.4204, -3.8669, -2.0628]])
tensor([[-1.0939,  2.4447, -0.6503, -1.4798,  2.0682],
        [-0.2408,  1.6893, -1.3638,  1.2721, -4.3678],
        [ 0.3614, -2.0208, -2.1008, -0.1139, -8.5612],
        [-2.4275, -5.0651,  3.9865, -0.0571, -6.5682],
        [-3.3370, -1.3500,  4.4204, -3.8669, -2.0628]])


tensor([[[[True, True, True, True, True],
          [True, True, True, True, True],
          [True, True, True, True, True],
          [True, True, True, True, True],
          [True, True, True, True, True]]]])